In [1]:
import json
import re
import nltk
import emoji
import numpy as np
import pandas as pd
from unicodedata import normalize
from wordcloud import WordCloud
from pysentimiento.preprocessing import preprocess_tweet

## Dataset: SENT-COVID

In [2]:
with open('data/SENT-COVID.json') as file:
    data = json.load(file)
    
pd.options.mode.chained_assignment = None                                         
pd.set_option('display.max_colwidth',None)   

df = pd.DataFrame(data)
df[['Tweet','clean_tweet','Label']].head(10)

,Tweet,clean_tweet,Label
0,1401047081121353728-@dulcema201 @BronstonRaqsa02 Protocolo de COVID !!!!,Protocolo de COVID !!!!,NEUTRO
1,1258159310162595843-#COVID19 #QuedateEnCasa en Morelia Centro,COVID QuedateEnCasa en Morelia Centro,NEUTRO
2,1272748988626862082-México va en en aumento con el #Covid_19. Tal vez no tengamos la estabilidad de Europa o estados unidos. Para mantener 120 días en paro total. Pero podemos ser precavidos al usar la #SanaDistancia,México va en en aumento con el Covid. Tal vez no tengamos la estabilidad de Europa o estados unidos. Para mantener días en paro total. Pero podemos ser precavidos al usar la SanaDistancia,POSITIVO
3,1349385638722883585-@sororavirus Creo en todo y nada. 💜,Creo en todo y nada. 💜,NEUTRO
4,1360615587114844161-@GobiernoMX había prometido 389,había prometido,NEGATIVO
5,1384487745033162755-Vacunación en Tonalá,Vacunación en Tonalá,POSITIVO
6,1335343584875327494-No es cierto lo que dice @WHO,No es cierto lo que dice,NEGATIVO
7,1257755579101003776-Participa en familia con esta entretenida actividad #MejorEnCasa Manda tu vídeo a la brevedad y llevaré uno de los premios.#aquiestamos #IED #QuedateEnCasa https://t.co/BQZTpxwca2,Participa en familia con esta entretenida actividad MejorEnCasa Manda tu vídeo a la brevedad y llevaré uno de los premios.aquiestamos IED QuedateEnCasa,POSITIVO
8,1317288163065401345-@lalitogonzaalez Acabar con el Coronavirus.,Acabar con el Coronavirus.,POSITIVO
9,1255373406624776194-#InternationalDanceDay #DID #ENCASA https://t.co/crIQOdoPgG,InternationalDanceDay DID ENCASA,NEUTRO


## Limpieza y Normalización

In [3]:
def clean_text(text):
  text = re.sub(r'^RT[\s]+', '', text) # RT's
  text = re.sub(r'https?:\/\/.*[\r\n]*', '', text) # Url's
  text = re.sub(r'-', '', text) #guiones
  text = re.sub(r'_', '', text) #guiones bajos 
  text = re.sub(r'@[A-Za-z0-9_]+', '', text) #menciones
  text = re.sub(r'[~^0-9]', '', text) #numeros
  text = re.sub("\\s+", ' ', text) ##Espacios blancos dobles
  text = re.sub('\n', ' ', text) ##Saltos de linea

  pattern = r'([.])([A-Z])'
  text = re.sub(pattern, r'\1 \2', text) # Separacion de punto seguido por una mayuscula
  return text

def normalize(text):    
   text = text.lower() # Se convierte todo el texto a minúsculas
   text = re.sub(r'#', '', text) # Eliminacion de almohadillas 
   text = re.sub('http\S+', ' ', text) # Eliminación de páginas web 
   text = re.sub('emoji', '', text) # Eliminación de palabra emoji
   regex = '[\\!\\"\\#\\$\\%\\&\\\'\\(\\)\\*\\+\\,\\-\\.\\/\\:\\;\\<\\=\\>\\?\\@\\[\\\\\\]\\^_\\`\\{\\|\\}\\~]'
   text = re.sub(regex , ' ', text) # Eliminacion de signos de puntuacion
   text = re.sub("\d+", ' ', text) # Eliminación de números
   text = re.sub("\\s+", ' ', text) # Eliminación de espacios en blanco múltiples
   #text = re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", normalize( "NFD"), 0, re.I)
   #text = normalize( 'NFC')
   return(text)

def tokenize(text):    
    text= text.split(sep = ' ')  # Tokenización por palabras individuales
    text= [token for token in text if len(token) > 1]  # Eliminación de tokens con una longitud < 2
    return(text)

def remove_punct(text):
 text = "".join(u for u in text if u not in ("?","¿", ".", ";", ":", "!","¡",'"',"%","“","”","$","&","'","\\",
                                                "*","+",",","/","<",">","=","^","•","...", "ç","π","ⓘ"))
 a,b = 'áéíóúÁÉÍÓÚ','aeiouAEIOU'
 trans = str.maketrans(a,b)     
 text = text.translate(trans) # Reemplazo de palabras acentuadas                                    
 return text

def remove_emoji(text):
  return emoji.get_emoji_regexp().sub(r'', text) # Eliminacion de emojis

def py_preprocess(text):
  return preprocess_tweet(text)  # Preprocesamiento de pysentimiento (opcional)

def split_hashtags(text):
  pat1 = r'([#])'
  text = re.sub(pat1, ' ', text) # Reemplazo de almohadillas por espacios en blanco
  pat2 = r'([a-z])([A-Z])'
  text = re.sub(pat2, r'\1 \2', text) # Separacion de minisculas seguidas por una mayuscula
  return text    

In [4]:
df['clean_tweet'] = df['Tweet'].apply(clean_text)
df['nopunct_tweet'] = df['clean_tweet'].apply(remove_punct)
df['noemoji_tweet'] = df['nopunct_tweet'].apply(remove_emoji)
#df['pypre_tweet'] = df['noemoji_tweet'].apply(py_preprocess)
df['nohash_tweet'] = df['noemoji_tweet'].apply(split_hashtags)
df['norm_tweet'] = df['nohash_tweet'].apply(normalize)
df['tokenized_tweet'] = df['norm_tweet'].apply(tokenize)
df[['Label', 'clean_tweet', 'norm_tweet','tokenized_tweet']][150:170]

/var/folders/z7/lg2xbgc16657k0q2mcpjssgw0000gn/T/ipykernel_795/1100998391.py:42: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  return emoji.get_emoji_regexp().sub(r'', text) # Eliminacion de emojis


,Label,clean_tweet,norm_tweet,tokenized_tweet
150,POSITIVO,razón por la que quiero que me dé covid:. donar plasma,razon por la que quiero que me de covid donar plasma,"[razon, por, la, que, quiero, que, me, de, covid, donar, plasma]"
151,NEUTRO,Si he leído bro,si he leido bro,"[si, he, leido, bro]"
152,NEUTRO,No sé si yo tomo las cosas muy directas o fue con otra intención tome un taxi el cual me hace a plática él conductor hablamos del COVID y de ahí pasamos a qué tenía piedras en el riñón ya que el bebía y que eso lo ponía muy caliente y que tenía que llegar a partir a su mujer,no se si yo tomo las cosas muy directas o fue con otra intencion tome un taxi el cual me hace a platica el conductor hablamos del covid y de ahi pasamos a que tenia piedras en el riñon ya que el bebia y que eso lo ponia muy caliente y que tenia que llegar a partir a su mujer,"[no, se, si, yo, tomo, las, cosas, muy, directas, fue, con, otra, intencion, tome, un, taxi, el, cual, me, hace, platica, el, conductor, hablamos, del, covid, de, ahi, pasamos, que, tenia, piedras, en, el, riñon, ya, que, el, bebia, que, eso, lo, ponia, muy, caliente, que, tenia, que, llegar, partir, su, mujer]"
153,POSITIVO,Los campos de golf pasarán a la historia en breve. Un uso energético sustentable y #agrovoltaico sería formidable. Fomentar economía circular,los campos de golf pasaran a la historia en breve un uso energetico sustentable y agrovoltaico seria formidable fomentar economia circular,"[los, campos, de, golf, pasaran, la, historia, en, breve, un, uso, energetico, sustentable, agrovoltaico, seria, formidable, fomentar, economia, circular]"
154,POSITIVO,Feliz martes!!#quedateencasa envíos a toda la República#stayhome...............#oaxaca #mexico #travel #swang #beach #lifeisgood #instapic #picoftheday #lifeisgood #love #instagood #nature…,feliz martes quedateencasa envios a toda la republica stayhome oaxaca mexico travel swang beach lifeisgood instapic picoftheday lifeisgood love instagood nature…,"[feliz, martes, quedateencasa, envios, toda, la, republica, stayhome, oaxaca, mexico, travel, swang, beach, lifeisgood, instapic, picoftheday, lifeisgood, love, instagood, nature…]"
155,NEGATIVO,⭕#AlertaCovid 🦠No es un juego... Tabasco acumuló nuevos casos de #COVID en las últimas horas,alerta covid no es un juego tabasco acumulo nuevos casos de covid en las ultimas horas,"[alerta, covid, no, es, un, juego, tabasco, acumulo, nuevos, casos, de, covid, en, las, ultimas, horas]"
156,NEUTRO,Para buen material sigan a mi amiga 👉🏻🍑 🔥 ¿Quieres ver un vídeo? Da click 👉🏻,para buen material sigan a mi amiga quieres ver un video da click,"[para, buen, material, sigan, mi, amiga, quieres, ver, un, video, da, click]"
157,NEUTRO,Ustedes perdonarán lo explícito 😋pero es necesario,ustedes perdonaran lo explicito pero es necesario,"[ustedes, perdonaran, lo, explicito, pero, es, necesario]"
158,NEGATIVO,"""¿Cómo puedes celebrar el #DiaInternacionalDeLaPaz? El tema está relacionado con la lucha contra la #Covid",como puedes celebrar el dia internacional de la paz el tema esta relacionado con la lucha contra la covid,"[como, puedes, celebrar, el, dia, internacional, de, la, paz, el, tema, esta, relacionado, con, la, lucha, contra, la, covid]"
159,NEGATIVO,"""Pésima política pública que generará un incremento sustancial en casos COVID en Jalisco el “🚨“. Protagonismo y drama.""",pesima politica publica que generara un incremento sustancial en casos covid en jalisco el protagonismo y drama,"[pesima, politica, publica, que, generara, un, incremento, sustancial, en, casos, covid, en, jalisco, el, protagonismo, drama]"


## Lemmatización 

In [5]:
import spacy
from nltk.stem.snowball import SnowballStemmer

sp = spacy.load('es_core_news_sm')

def lemmatization(text):
    doc = sp(text)
    return ' '.join([word.lemma_ for word in doc]) 

#stemmer = SnowballStemmer('spanish')
#stemmed_spanish = [stemmer.stem(item) for item in spanish_words]

In [6]:
df['lem_tweet'] = df['norm_tweet'].apply(lemmatization)
df['lem_tweet'] = df['lem_tweet'].apply(remove_punct)
df['lemtokenized_tweet'] = df['lem_tweet'].apply(tokenize)
df[['Label', 'norm_tweet','lem_tweet','tokenized_tweet','lemtokenized_tweet']][200:220]

,Label,norm_tweet,lem_tweet,tokenized_tweet,lemtokenized_tweet
200,POSITIVO,dos años con rumbo sntesalud y corona tips se han convertido en referente y aporte snte en la lucha ante la covid en una accion de corresponsabilidad social con el snte al uno somos todos todos somos uno ver documento,dos año con rumbo sntesalud y corona tip el haber convertir en referente y aporte snte en el lucha ante el covid en uno accion de corresponsabilidad social con el snte al uno ser todo todo ser uno ver documento,"[dos, años, con, rumbo, sntesalud, corona, tips, se, han, convertido, en, referente, aporte, snte, en, la, lucha, ante, la, covid, en, una, accion, de, corresponsabilidad, social, con, el, snte, al, uno, somos, todos, todos, somos, uno, ver, documento]","[dos, año, con, rumbo, sntesalud, corona, tip, el, haber, convertir, en, referente, aporte, snte, en, el, lucha, ante, el, covid, en, uno, accion, de, corresponsabilidad, social, con, el, snte, al, uno, ser, todo, todo, ser, uno, ver, documento]"
201,POSITIVO,reglasdeoro numero cuida tu salud feliz dimingo quedateen casa buen domingo sana distancia juntosporla salud xalapa devozenvozmx feliz dia del padre,reglasdeoro numero cuidar tu salud feliz dimingo quedateen casar buen domingo sano distancia juntospor el salud xalapa devozenvozmx feliz dia del padre,"[reglasdeoro, numero, cuida, tu, salud, feliz, dimingo, quedateen, casa, buen, domingo, sana, distancia, juntosporla, salud, xalapa, devozenvozmx, feliz, dia, del, padre]","[reglasdeoro, numero, cuidar, tu, salud, feliz, dimingo, quedateen, casar, buen, domingo, sano, distancia, juntospor, el, salud, xalapa, devozenvozmx, feliz, dia, del, padre]"
202,POSITIVO,en coahuila se reforzaran las medidas para que se respeten los protocolos sanitarios y prevenir un posible incremento en los contagios por covid por la temporada vacacional y ante el repunte de la pandemia que se registra en el pais,en coahuila el reforzar el medida para que el respetar el protocolo sanitario y prevenir uno posible incremento en el contagio por covid por el temporada vacacional y ante el repunte de el pandemia que el registrar en el pais,"[en, coahuila, se, reforzaran, las, medidas, para, que, se, respeten, los, protocolos, sanitarios, prevenir, un, posible, incremento, en, los, contagios, por, covid, por, la, temporada, vacacional, ante, el, repunte, de, la, pandemia, que, se, registra, en, el, pais]","[en, coahuila, el, reforzar, el, medida, para, que, el, respetar, el, protocolo, sanitario, prevenir, uno, posible, incremento, en, el, contagio, por, covid, por, el, temporada, vacacional, ante, el, repunte, de, el, pandemia, que, el, registrar, en, el, pais]"
203,POSITIVO,tenemos a la venta cubreboca de neopreno de primera calidad somos fabricantes pregunta por nuestros precios mayoreo y menudeo promoales cuarentena covidmx cubrebocas,tener a el venta cubreboco de neopreno de primero calidad ser fabricant preguntar por nuestro precio mayoreo y menudeo promoal cuarenten covidmx cubreboca,"[tenemos, la, venta, cubreboca, de, neopreno, de, primera, calidad, somos, fabricantes, pregunta, por, nuestros, precios, mayoreo, menudeo, promoales, cuarentena, covidmx, cubrebocas]","[tener, el, venta, cubreboco, de, neopreno, de, primero, calidad, ser, fabricant, preguntar, por, nuestro, precio, mayoreo, menudeo, promoal, cuarenten, covidmx, cubreboca]"
204,NEUTRO,y apoco usted duda de todo nunca deseo el mal a nadie,y apoco usted dudar de todo nunca desear el mal a nadie,"[apoco, usted, duda, de, todo, nunca, deseo, el, mal, nadie]","[apoco, usted, dudar, de, todo, nunca, desear, el, mal, nadie]"
205,NEGATIVO,yo no tengo el placer de conocerla en carne y hueso,yo no tener el placer de conocer el en carne y hueso,"[yo, no, tengo, el, placer, de, conocerla, en, carne, hueso]","[yo, no, tener, el, placer, de, conocer, el, en, carne, hueso]"
206,POSITIVO,la magia de la radio antes de la covid en exa queretaro,el magia de el radio antes de el covid en exa queretaro,"[la, ma

## Nltk Stopwords

In [7]:
from nltk.corpus import stopwords

# Obtención de listado de stopwords del español
stop_words_esp = list(stopwords.words('spanish'))
print(stop_words_esp[:100])

['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosotros', 'vosotras', 'os', 'mío', 'mía', 'míos', 'mías', 'tuyo']


## Train-Test Split

In [10]:
from sklearn.model_selection import train_test_split

X1 = df['norm_tweet']
X2 = df['lem_tweet']
y = df['Label']

# Separamos el corupus 1) con columna 'norm_tweet' 2) con columna 'lem_tweet'
#X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.25 ,random_state=37)
X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.25 ,random_state=37)

In [11]:
value, counts = np.unique(y_train, return_counts=True)
print(dict(zip(value, 100 * counts / sum(counts))))
value, counts = np.unique(y_test, return_counts=True)
print(dict(zip(value, 100 * counts / sum(counts))))

{'NEGATIVO': 33.7124526100904, 'NEUTRO': 44.911052785068534, 'POSITIVO': 21.37649460484106}
{'NEGATIVO': 35.345581802274715, 'NEUTRO': 43.39457567804025, 'POSITIVO': 21.25984251968504}


## Vectorizaciones


### TFIDF

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf= TfidfVectorizer(min_df= 3, ngram_range=(1,2), stop_words = stop_words_esp).fit(X_train)
                        
print('Numero de features: ' +str(len(tfidf.get_feature_names_out())))
tfidf.fit(X_train)

Numero de features: 2291


TfidfVectorizer(min_df=3, ngram_range=(1, 2),
                stop_words=['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los',
                            'del', 'se', 'las', 'por', 'un', 'para', 'con',
                            'no', 'una', 'su', 'al', 'lo', 'como', 'más',
                            'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí',
                            'porque', ...])

In [13]:
# Creacion de la matriz 
X_train_tfidf = tfidf.transform(X_train)
X_test_tfidf  = tfidf.transform(X_test)
X_train_tfidf 

<3429x2291 sparse matrix of type '<class 'numpy.float64'>'
	with 23705 stored elements in Compressed Sparse Row format>

### CountVectorizer

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

countvect = CountVectorizer(min_df=3, stop_words = stop_words_esp, ngram_range=(1,2)).fit(X_train)

print('Numero de features: ' +str(len(countvect.get_feature_names_out())))
countvect.fit(X_train)

Numero de features: 2291


CountVectorizer(min_df=3, ngram_range=(1, 2),
                stop_words=['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los',
                            'del', 'se', 'las', 'por', 'un', 'para', 'con',
                            'no', 'una', 'su', 'al', 'lo', 'como', 'más',
                            'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí',
                            'porque', ...])

In [15]:
# Creacion de la matriz
X_train_cv = countvect.transform(X_train)
X_test_cv  = countvect.transform(X_test)
X_train_cv

<3429x2291 sparse matrix of type '<class 'numpy.int64'>'
	with 23705 stored elements in Compressed Sparse Row format>

### Vocabulario 

In [16]:
# vocabuilario TFIDF
print(f" Número de tokens creados: {len(tfidf.get_feature_names_out())}")
tfidf.get_feature_names()

# vocabulario CountVectorizer
#print(f" Número de tokens creados: {len(countvect.get_feature_names_out())}")
#countvect.get_feature_names()

 Número de tokens creados: 2291


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['aaa',
 'abierto',
 'abrazo',
 'abril',
 'abrir',
 'abuelita',
 'abuelito',
 'aburrido',
 'aburrir',
 'aca',
 'acabar',
 'acabar cuarentena',
 'acabar pandemia',
 'acapulco',
 'acceso',
 'accidente',
 'accion',
 'aceptar',
 'acerca',
 'aclas',
 'acompañar',
 'acordar',
 'acostumbrar',
 'actividad',
 'activo',
 'activo covid',
 'actual',
 'actualizacion',
 'actuar',
 'acudir',
 'acuerdo',
 'adecuado',
 'adelante',
 'adema',
 'administracion',
 'adomicilio',
 'adquirir',
 'adulto',
 'adulto mayor',
 'aeropuerto',
 'afectar',
 'agarrar',
 'agosto',
 'agradecer',
 'agua',
 'aguascalient',
 'ah',
 'ahh',
 'ahi',
 'ahora',
 'ahora haber',
 'ahora resultar',
 'ahora si',
 'ahoritir',
 'aire',
 'aire ılılı',
 'aislamiento',
 'alcoholico',
 'alegrar',
 'alejandro',
 'alemania',
 'alergia',
 'alerta',
 'alerta maximo',
 'alfonso',
 'alguien',
 'alguien decir',
 'alguien haber',
 'alguien mas',
 'alguien saber',
 'alguien tener',
 'algun',
 'algun dia',
 'alguno',
 'allo',
 'alrededor',
 'alto',

## Naive Bayes

In [19]:
from sklearn import naive_bayes
from sklearn import metrics

nb = naive_bayes.MultinomialNB()
nb.fit(X_train_cv, y_train)

predictions_nb = nb.predict(X_test_cv)
metrics.f1_score(y_test, predictions_nb, average='micro')

0.5818022747156606

In [20]:
print(nb.score(X_train_cv,y_train))
print(nb.score(X_test_cv,y_test))
print(nb.classes_)
predictions_nb

0.7471566054243219
0.5818022747156606
['NEGATIVO' 'NEUTRO' 'POSITIVO']


array(['POSITIVO', 'NEUTRO', 'NEGATIVO', ..., 'NEGATIVO', 'NEUTRO',
       'NEGATIVO'], dtype='<U8')

In [21]:
from sklearn.metrics import confusion_matrix,classification_report

new = np.asarray(y_test)
confusion_matrix(predictions_nb,y_test)
print(classification_report(predictions_nb, y_test))

              precision    recall  f1-score   support

    NEGATIVO       0.68      0.58      0.63       472
      NEUTRO       0.59      0.60      0.60       488
    POSITIVO       0.40      0.53      0.46       183

    accuracy                           0.58      1143
   macro avg       0.56      0.57      0.56      1143
weighted avg       0.60      0.58      0.59      1143

